In [1]:
from utils import *
import glob
sns.set_style('whitegrid')


T=int(1e3)
ts = list(range(1,T+1))
gamma=4

In [2]:
from itertools import product


mps, n_gossips = [f"baseline", f"Flooding-Absorption", f"Flooding", f"Flooding-RandomStop-0.5"], [1, None]
# collect datas in parallel
partial_params = list(product(mps, n_gossips))
# run experiment only once for baseline
partial_params = [item for item in partial_params if
                  item[0] != "baseline" or (item[0] == "baseline" and item[1] is None)]
# remove Hitting+Gossiping
partial_params = [item for item in partial_params if "Absorption" not in item[0] or item[1] is None]
# remove RS+Gossiping
partial_params = [item for item in partial_params if "RandomStop" not in item[0] or item[1] is None]

print(partial_params)

[('baseline', None), ('Flooding-Absorption', None), ('Flooding', 1), ('Flooding', None), ('Flooding-RandomStop-0.5', None)]


In [3]:
## new plots for regret/comm complexity vs. t
# legends=None
# legends = ["baseline", "Flooding-A", "gossiping", "Flooding", "Prob Flooding (0.5)", "IRS"]
legends = ["baseline", "Flooding-A", "gossiping", "Flooding", "Prob Flooding (0.5)", "IRS"]
# for dynamic in [""]:
for dynamic in ["_dynamic_sparse"]:
    if dynamic == "":
        RGs = ['ER', 'BA', 'SBM']
    else:
        RGs = ['ER']
    for RG in RGs:
        path = f"results-uniform_N_100_K_50_k_20/heterogeneous_K=50{dynamic}/{RG}"
        for mode in ["Regret", "Communication", "Message"]:
            names2 = glob.glob(f"{path}/{mode}_final_t_p=1.0_gamma={gamma}*.npz")
            tmp2 = np.load(names2[0])
            mean, std = tmp2['arr_0'], tmp2['arr_1']
            # if mean.shape[0] > 4:
            #     mean, std = np.delete(mean, 1, 0), np.delete(std, 1, 0)
            
            # overlay results for IRS (Flooding with gamma=1, no gossip)
            names = glob.glob(f"{path}/{mode}_final_t_p=1.0_gamma=1*.npz")
            tmp1 = np.load(names[0])
            mean1, std1 = tmp1['arr_0'], tmp1['arr_1']
            mean = np.concatenate((mean, mean1[-1].reshape((1,T))))
            std = np.concatenate((std, std1[-1].reshape((1,T))))
            # # overlay results for naive stopping
            # regret_naive = np.load(f"{path}/{RG_model_name}_naive/Regret_final_t_p=1.0_gamma=2_{RG_model}.npz")
            # regret_mean_naive, regret_std_naive = regret_naive['arr_0'], regret_naive['arr_1']
        
            title = f"{mode} vs. t ({RG}, gamma={gamma}, {dynamic})"
            fname = f"{path}/{mode}_final_t_plot.pdf"
        
            plot_final(mean, std, ts, title, "t", fname, legends)

In [10]:
## redo plots for gamma experiments
gammas = [1, 2, 3, 4]
for RG_model in RG_models:
    if RG_model not in ['Path', 'Star', 'Cycle']:
        RG_model_name = RG_model[:RG_model.index('_')]
    else:
        RG_model_name = RG_model

    legends = None
    # legends = ["baseline", "Flooding (no gossip)", "Flooding (gossip)", "Flooding-A"]

    # plot regret
    regret = np.load(f"{path}/{RG_model_name}/Regret_final_gamma_p=1.0_{RG_model}.npz")
    regret_mean, regret_std = regret['arr_0'], regret['arr_1']
    regret_mean, regret_std = regret_mean, regret_std

    regret_title = f"[Regret] Varying gamma ({RG_model_name})"
    regret_fname = f"{path}/final_plots/Regret_final_gamma_p=1.0_{RG_model}.pdf"

    plot_final_discrete(regret_mean, regret_std, gammas, regret_title, "gamma", regret_fname, legends)

    # plot communication
    communication = np.load(f"{path}/{RG_model_name}/Communication_final_gamma_p=1.0_{RG_model}.npz")
    communication_mean, communication_std = communication['arr_0'], communication['arr_1']
    communication_mean, communication_std = communication_mean, communication_std

    communication_title = f"[Communication] Varying gamma ({RG_model_name})"
    communication_fname = f"{path}/final_plots/Communication_final_gamma_p=1.0_{RG_model}.pdf"

    plot_final_discrete(communication_mean, communication_std, gammas, communication_title, "gamma", communication_fname, legends)

FileNotFoundError: [Errno 2] No such file or directory: 'results/heterogeneous_K=40/Star/Regret_final_gamma_p=1.0_Star.npz'

In [ ]:
## new plots for regret/comm complexity vs. p
ps = list(np.linspace(0, 1.0, num=21))
P = len(ps)
for RG_model in RG_models:
    if RG_model not in ['Path', 'Star', 'Cycle']:
        RG_model_name = RG_model[:RG_model.index('_')]
    else:
        RG_model_name = RG_model

    legends=None
    # legends = ["baseline", "Flooding (gamma=2, no gossip)", "Flooding (gamma=2, gossip)", "Flooding-A", "Flooding (gamma=1, no gossip)"]

    # plot regret
    regret_2 = np.load(f"{path}/{RG_model_name}/Regret_final_p_gamma=2_{RG_model}.npz")
    regret_mean, regret_std = regret_2['arr_0'], regret_2['arr_1']
    regret_mean, regret_std = regret_mean[:-1], regret_std[:-1]
    # overlay results for IRS (Flooding with gamma=1, no gossip)
    regret_1 = np.load(f"{path}/{RG_model_name}/Regret_final_p_gamma=1_{RG_model}.npz")
    regret_mean1, regret_std1 = regret_1['arr_0'], regret_1['arr_1']

    regret_mean = np.concatenate((regret_mean, regret_mean1[0].reshape((1,P))))
    regret_std = np.concatenate((regret_std, regret_std1[0].reshape((1,P))))

    regret_title = f"Regret vs. p ({RG_model_name})"
    regret_fname = f"{path}/final_plots/Regret_final_p_{RG_model}.pdf"

    plot_final(regret_mean, regret_std, ps, regret_title, "p", regret_fname, legends)

    # plot communication
    communication_2 = np.load(f"results/heterogeneous_K=20/{RG_model_name}/Communication_final_p_gamma=2_{RG_model}.npz")
    communication_mean, communication_std = communication_2['arr_0'], communication_2['arr_1']
    communication_mean, communication_std = communication_mean[:-1], communication_std[:-1]
    # overlay results for IRS (Flooding with gamma=1, no gossip)
    communication_1 = np.load(f"{path}/{RG_model_name}/Communication_final_p_gamma=1_{RG_model}.npz")
    communication_mean1, communication_std1 = communication_1['arr_0'], communication_1['arr_1']

    communication_mean = np.concatenate((communication_mean, communication_mean1[0].reshape((1,P))))
    communication_std = np.concatenate((communication_std, communication_std1[0].reshape((1,P))))

    communication_title = f"Communication vs. p ({RG_model_name})"
    communication_fname = f"{path}/final_plots/Communication_final_p_{RG_model}.pdf"

    plot_final(communication_mean, communication_std, ps, communication_title, "p", communication_fname, legends)

In [ ]:
## new metric
# legends=None
# legends = ["baseline", "Flooding (gamma=2, no gossip)", "gossiping", "Flooding-A", "IRS", "naive-stopping"]
path = f"results-uniform_N_100_K_50_k_20/heterogeneous_K=50/SBM"
metrics1 = np.ones((4,T))
metrics2 = np.ones((4,T))
eps = 1e-1
for mode in ["Regret", "Communication"]:
    names2 = glob.glob(f"{path}/{mode}_final_t_p=1.0_gamma={gamma}.npz")
    tmp2 = np.load(names2[0])
    mean, std = tmp2['arr_0'], tmp2['arr_1']
    if mean.shape[0] > 4:
        mean, std = np.delete(mean, 1, 0), np.delete(std, 1, 0)
    
    if mode == "Regret":
        tmp = mean[0] - mean
        metrics2 *= tmp
    else:
        tmp = np.log(mean)
        metrics2 /= tmp

    if mode == "Communication":
        # metrics1 *= mean + 1
        metrics1 *= np.log(mean + 2)
    else:
        metrics1 *= mean

In [ ]:
title = f"metric vs. t (ER, gamma={gamma})"
fname = f"{path}/metric_final_t_plot.pdf"

legends = ["baseline", "Flooding-A", "gossiping", "Flooding"]

fig, ax = plt.subplots()
clrs = sns.color_palette("husl", 4)

xs = range(1, T+1)
with sns.axes_style("whitegrid"):
    for i, color in enumerate(clrs):
        ax.plot(xs, metrics1[i], label=legends[i], c=color)

ax.set_title(title)
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

title = (f"metric vs. t (ER, gamma={gamma})")
fname = f"{path}/metric_final_t_plot.pdf"

legends = ["baseline", "Flooding-A", "gossiping", "Flooding"]
# legends = ["baseline", "Flooding-A", "gossiping"]

fig, ax = plt.subplots()
clrs = sns.color_palette("husl", 3)

xs = range(1, T+1)
with sns.axes_style("whitegrid"):
    for i, color in enumerate(clrs):
        ax.plot(xs, metrics2[i+1], label=legends[i+1], c=color)

ax.set_title(title)
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()